In [ ]:
from IPython import display
import torch
import sys
from torch import nn, optim
from torch.autograd.variable import Variable
from torchvision import transforms, datasets
import torch.nn.functional as F
from torchvision.utils import save_image
content_folder = './'

img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,)),
])


train_data = datasets.MNIST(
    root='./content/mnist',
    train=True,
    download=True,
    transform=img_transform
)

test_data = datasets.MNIST(
    root='./content/mnist',
    train=False,
    download=True,
    transform=img_transform
)

train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=100, shuffle=False)

In [ ]:
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
        super(VAE, self).__init__()
        
        # encoder layers
        self.fc1 =  nn.Sequential( 
              nn.Linear(x_dim, 1024),
              nn.LeakyReLU(0.2),
              nn.Dropout(0.3)
            )
        self.fc2 =  nn.Sequential( 
              nn.Linear(1024, h_dim1),
              nn.LeakyReLU(0.2),
              nn.Dropout(0.3)
            )
        self.fc3 =nn.Sequential(  
              nn.Linear(h_dim1, h_dim2),
              nn.LeakyReLU(0.2),
              nn.Dropout(0.3)
        )
        self.fc41 = nn.Sequential(  
              nn.Linear(h_dim2, z_dim),
              nn.LeakyReLU(0.2),
              nn.Dropout(0.3)
            )
        self.fc42 = nn.Sequential(  
              nn.Linear(h_dim2, z_dim),
              nn.LeakyReLU(0.2),
              nn.Dropout(0.3)
            )
        
        # decoder layers
        self.fc5 =  nn.Sequential( 
              nn.Linear(z_dim, h_dim2),
              nn.LeakyReLU(0.2),
            )
        self.fc6 = nn.Sequential(
              nn.Linear(h_dim2, h_dim1),
              nn.LeakyReLU(0.2)
            ) 
        self.fc7 =  nn.Sequential(
              nn.Linear(h_dim1, 1024),
              nn.LeakyReLU(0.2)
            )
        self.fc8 =  nn.Sequential(
              nn.Linear(1024, x_dim),
              nn.LeakyReLU(0.2))
        
    def encoder(self, x):
        h = self.fc1(x)
        h = self.fc2(h)
        h = self.fc3(h)
        return self.fc41(h), self.fc42(h) # mean, log of variance
    
    def sampling(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu) # return a latent vector sample z from approx prior
        
    def decoder(self, z):
        h = self.fc5(z)
        h = self.fc6(h)
        h = self.fc7(h)
        h = self.fc8(h)
        return torch.sigmoid(h) 
    
    def forward(self, x):
        mu, log_var = self.encoder(x)
        z = self.sampling(mu, log_var)
        return self.decoder(z), mu, log_var

vae = VAE(x_dim=784, h_dim1= 512, h_dim2=256, z_dim=2)  

In [ ]:
def loss_function(recon_x, x, mu, log_var):
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD

def train_VAE(optimizer, data):
    
    optimizer.zero_grad()
    recon_batch, mu, log_var = vae(data)
    error = loss_function(recon_batch, data, mu, log_var)
    error.backward()
    optimizer.step()
    return error
    
def img_to_vector(img):
    return img.view(img.size(0), 784)

def vector_to_img(vector):
    return vector.view(vector.size(0), 1, 28, 28)    


In [ ]:
v_optimizer = optim.Adam(vae.parameters(), lr=0.0002)
num_epochs = 300
num_test_samples = 64

for epoch in range(num_epochs):
    vae.train()
    train_loss = 0
    for n_batch, (real_batch,_) in enumerate(train_loader):
        real_data = Variable(img_to_vector(real_batch))
        loss = train_VAE(v_optimizer,real_data)
        train_loss += loss.item()
        
        
        if n_batch % 100 == 0:
            display.clear_output(True)
            z = torch.randn(num_test_samples, 2)
            sample = vae.decoder(z)
            test_images = vector_to_img(sample).data.cpu()
            save_image(test_images, f"./content/img_{epoch}_{n_batch}.png", normalize=True)
                        
    vae.eval()
    test_loss= 0
    with torch.no_grad():
        for data, _ in test_loader:
            data = data
            real_data = Variable(img_to_vector(real_batch))
            recon, mu, log_var = vae(real_data)
            
            test_loss += loss_function(recon, real_data, mu, log_var).item()
        
    test_loss /= len(test_loader.dataset)
    print('Loss for test data: {:.8f}'.format(test_loss))